In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import models,layers
from keras import regularizers
from sklearn.model_selection import train_test_split


In [16]:
df = pd.read_csv('./creditcard.csv')
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4656,4046,-1.419157,1.751823,0.871490,0.033815,-0.374373,-0.166273,-0.248420,0.966657,0.390515,-0.715362,2.297797,-1.130493,2.297093,2.243270,-0.464764,0.271479,0.659115,-0.046540,-0.301317,-0.022924,-0.239606,-0.551645,0.109464,-0.049862,-0.202307,0.052414,0.119387,0.026361,8.99,0.0
4657,4048,-1.619060,0.687492,2.283881,0.135850,-0.348660,2.173278,-0.637450,1.274966,1.318315,-1.127718,2.634005,-1.659902,0.535918,1.684505,-0.214868,-0.960104,1.851847,-1.146547,-1.351567,-0.235185,-0.018311,0.387256,0.109870,-0.870232,-0.150190,0.425930,0.113039,0.013690,60.90,0.0
4658,4049,-0.677034,-0.031392,2.591247,-0.756693,-0.266149,0.364260,0.201486,0.074833,1.780884,-1.245756,2.193685,-1.196707,1.809113,0.648516,-2.222445,-0.161459,0.541773,-0.385646,-0.612098,0.079597,-0.230477,-0.203616,0.077837,0.224800,-0.213579,0.713807,-0.246913,-0.229547,69.34,0.0
4659,4053,1.104345,-0.531407,1.088893,0.307874,-1.049402,0.166874,-0.869630,0.193954,2.418299,-0.641927,1.363010,-1.948131,0.472848,1.289907,-1.587233,0.203006,0.570556,0.385682,0.670404,-0.076194,-0.339354,-0.664476,0.011053,-0.004883,0.072565,0.905579,-0.080857,0.000137,55.07,0.0


In [17]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)

x_data= data.drop(columns="Class", axis=0)
label = data["Class"]

In [18]:
train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [19]:
train_mean = train_data.mean(axis=0)
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [20]:
model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [21]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [22]:
model.fit(train_data, train_labels, epochs=100,validation_split = 0.3)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 1.0321 - accuracy: 0.3333 - val_loss: 1.0752 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 41ms/step - loss: 0.8223 - accuracy: 0.3333 - val_loss: 1.0677 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 43ms/step - loss: 1.4871 - accuracy: 0.3333 - val_loss: 1.0515 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 46ms/step - loss: 0.9064 - accuracy: 0.3333 - val_loss: 1.0478 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 41ms/step - loss: 0.6302 - accuracy: 0.6667 - val_loss: 1.0464 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 43ms/step - loss: 3.6657 - accuracy: 0.6667 - val_loss: 1.0349 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 45ms/step - loss: 1.2456 - accuracy: 0.3333 - val_loss: 1.0337 - val_accuracy: 0.000

In [23]:
loss, accuracy = model.evaluate(test_data, test_labels)

1/1 [==============================] - 0s 15ms/step - loss: 0.8131 - accuracy: 0.6667


In [24]:
loss

0.8130545020103455

In [25]:
accuracy

0.6666666865348816

In [30]:
x = model.predict(test_data).astype(dtype="u8")
a=x
b= np.array(test_labels)
a= a.reshape(1,a.shape[0])
np.size(a != b) - np.count_nonzero(a == b)

1

In [31]:
con = tf.math.confusion_matrix(labels= test_labels, predictions=x )
con

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 0],
       [1, 0]], dtype=int32)>